In [179]:
import json
import pandas as pd

In [180]:
configuration_path = "data/lyon_configuration.json"
scenario_path = "input/scenario_baseline_2022.json"
solution_path = "input/solution_baseline_2022.json"
output_path = "output/copert.xlsx"
year = 2022

## Read data

In [181]:
with open(solution_path) as f:
    solution = json.load(f)

In [182]:
with open(scenario_path) as f:
    scenario = json.load(f)

In [183]:
with open(configuration_path) as f:
    configuration = json.load(f)

## Prepare general data

In [184]:
months = [
    "January", "February", "March", "April", "May", "June", "July", 
    "August", "September", "October", "November", "December"]

In [185]:
df_min_temperature = pd.DataFrame.from_records([
    { "Month": month, str(year): temperature }
    for temperature, month in zip(configuration["temperature"]["min"], months)
])

df_max_temperature = pd.DataFrame.from_records([
    { "Month": month, str(year): temperature }
    for temperature, month in zip(configuration["temperature"]["max"], months)
])

In [186]:
df_humidity = pd.DataFrame.from_records([
    { "Month": month, str(year): humidity }
    for humidity, month in zip(configuration["humidity"], months)
])

In [187]:
df_sheets = pd.DataFrame.from_records([
    { "SHEET_NAME": "STOCK", "Unit": "[n]" },
    { "SHEET_NAME": "MEAN_ACTIVITY", "Unit": "[km]" },
    { "SHEET_NAME": "URBAN_OFF_PEAK_SPEED", "Unit": "[km/h]" },
    { "SHEET_NAME": "URBAN_PEAK_SPEED", "Unit": "[km/h]" },
    { "SHEET_NAME": "URBAN_OFF_PEAK_SHARE", "Unit": "[%]" },
    { "SHEET_NAME": "URBAN_PEAK_SHARE", "Unit": "[%]" },
    { "SHEET_NAME": "MIN_TEMPERATURE", "Unit": "[°C]" },
    { "SHEET_NAME": "MAX_TEMPERATURE", "Unit": "[°C]" },
    { "SHEET_NAME": "HUMIDITY", "Unit": "[%]" },
    { "SHEET_NAME": "FUEL_TANK_SIZE", "Unit": "[l]" },
])

## By vehicle type

In [188]:
vehicle_speeds = {}
vehicle_counts = {}
vehicle_distances = {}

for vt in scenario["vehicle_types"]:
    vehicle_speeds[vt["id"]] = vt["speed_km_h"]
    vehicle_counts[vt["id"]] = 0
    vehicle_distances[vt["id"]] = 0.0
    
for route in solution["routes"]:
    vehicle_counts[route["vehicle_type"]] += 1
    vehicle_distances[route["vehicle_type"]] += route["distance_km"]

In [189]:
df_stock = []
df_mean_activity = []

df_peak_speed = []
df_offpeak_speed = []

df_peak_share = []
df_offpeak_share = []

df_fuel_tank_size = []

In [190]:
for vt, config in configuration["vehicle_type_mapping"].items():
    assert vt in vehicle_counts
    
    common = {
        "Category": config["category"],
        "Fuel": config["fuel"],
        "Segment": config["segment"],
        "Euro Standard": config["euro_standard"]
    }
    
    peak_share = {}
    peak_share.update(common)
    peak_share[str(year)] = configuration["peak_share"] * 100.0
    df_peak_share.append(peak_share)
    
    offpeak_share = {}
    offpeak_share.update(common)
    offpeak_share[str(year)] = (1.0 - configuration["peak_share"]) * 100.0
    df_offpeak_share.append(offpeak_share)
    
    peak_speed = {}
    peak_speed.update(common)
    peak_speed[str(year)] = vehicle_speeds[vt] * configuration["peak_speed_factor"]
    df_peak_speed.append(peak_speed)
    
    offpeak_speed = {}
    offpeak_speed.update(common)
    offpeak_speed[str(year)] = vehicle_speeds[vt]
    df_offpeak_speed.append(offpeak_speed)
    
    fuel_tank_size = {}
    fuel_tank_size.update(common)
    fuel_tank_size[str(year)] = config["fuel_tank_size"]
    df_fuel_tank_size.append(fuel_tank_size)
    
    stock = {}
    stock.update(common)
    stock[str(year)] = vehicle_counts[vt]
    df_stock.append(stock)
    
    mean_activity = {}
    mean_activity.update(common)
    mean_activity[str(year)] = vehicle_distances[vt]
    df_mean_activity.append(mean_activity)

In [191]:
df_peak_share = pd.DataFrame.from_records(df_peak_share)
df_offpeak_share = pd.DataFrame.from_records(df_offpeak_share)
df_peak_speed= pd.DataFrame.from_records(df_peak_speed)
df_offpeak_speed = pd.DataFrame.from_records(df_offpeak_speed)
df_fuel_tank_size = pd.DataFrame.from_records(df_fuel_tank_size)
df_stock = pd.DataFrame.from_records(df_stock)
df_mean_activity = pd.DataFrame.from_records(df_mean_activity)

## Output

In [192]:
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    df_sheets.to_excel(writer, index = False, sheet_name = "SHEETS")
    df_stock.to_excel(writer, index = False, sheet_name = "STOCK")
    df_mean_activity.to_excel(writer, index = False, sheet_name = "MEAN_ACTIVITY")
    df_offpeak_speed.to_excel(writer, index = False, sheet_name = "URBAN_OFFPEAK_SPEED")
    df_peak_speed.to_excel(writer, index = False, sheet_name = "URBAN_PEAK_SPEED")
    df_offpeak_share.to_excel(writer, index = False, sheet_name = "URBAN_OFFPEAK_SHARE")
    df_peak_share.to_excel(writer, index = False, sheet_name = "URBAN_PEAK_SHARE")
    df_min_temperature.to_excel(writer, index = False, sheet_name = "MIN_TEMPERATURE")
    df_max_temperature.to_excel(writer, index = False, sheet_name = "MAX_TEMPERATURE")
    df_humidity.to_excel(writer, index = False, sheet_name = "HUMIDITY")
    df_fuel_tank_size.to_excel(writer, index = False, sheet_name = "FUEL_TANK_SIZE")